In [ ]:
import os
gpu_num = 0
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#os.environ['DRJIT_LIBLLVM_PATH'] = '/home/usare/.conda/envs/sionnasim/lib/libLLVM-17.so'

In [ ]:
import numpy as np
import tensorflow as tf
    
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera
from sionna.channel import subcarrier_frequencies, cir_to_ofdm_channel
%matplotlib widget
import matplotlib.pyplot as plt

In [ ]:
scene = load_scene("./scenes/scene1.xml")

In [ ]:
for i, obj in enumerate(scene.objects.values()):
    print(f"{obj.name} : {obj.radio_material.name}")
    if i >= 10:
        break

In [ ]:
scene.tx_array = PlanarArray(
                                num_rows=4,
                                num_cols=4,
                                vertical_spacing=0.5,
                                horizontal_spacing=0.5,
                                pattern="tr38901",
                                polarization="V"
                            )
scene.rx_array = PlanarArray(
                                num_rows=1,
                                num_cols=1,
                                vertical_spacing=0.5,
                                horizontal_spacing=0.5,
                                pattern="iso",
                                polarization="V"
                )


tx = Transmitter("tx", [-262, 192, 65], [0.0, 0.0, 0.0])
scene.add(tx)

rx = Receiver("rx", [-263, 166, 2], [0.0, 0.0, 0.0])
scene.add(rx)

rx2 = Receiver("rx2", [-320, 166, 2], [0.0, 0.0, 0.0])
scene.add(rx2)

In [ ]:
cm = scene.coverage_map()

In [ ]:
#scene.preview(coverage_map=cm)

In [ ]:
paths = scene.compute_paths(diffraction=True,
                                max_depth=3)
scene.preview(paths=paths)

In [ ]:
a, tau = paths.cir()

plt.figure()
plt.title("Channel impulse response")

t = tau[0,0,0,:]
a_bs = np.abs(a)[0,0,0,0,0,:,0]
plt.stem(t, a_bs)
plt.xlim([0, np.max(t)])
plt.xlabel("tau")
plt.ylabel("a")

In [ ]:
frequencies = subcarrier_frequencies(216*12, 15e3)
h_frequency = cir_to_ofdm_channel(frequencies, a, tau)

# Transformando em db
h_frequency_db = 20*np.log10(tf.abs(h_frequency).numpy())

plt.figure()
plt.plot(frequencies, h_frequency_db[0,0,0,0,0, 0,:])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB)')
plt.grid(True)
plt.show()

In [ ]:
scene.objects
scene.add(Camera("cam", position=[-500,240,45], look_at=[0,50,0], ))
car_01 = scene.get("car_01")
car_02 = scene.get("car_02")
# Put rx and rx1 in the same position as the cars
scene.get("rx").position = (car_01.position + [0, 0, 2])
scene.get("rx2").position = (car_02.position + [0, 0, 2])
# For n steps, move the cars forward (-X)
num_steps = 5
for i in range(num_steps):
    paths = scene.compute_paths(diffraction=True,
                                max_depth=3)
    scene.render(camera="cam", paths=paths)
    car_01.position -= [10, 0, 0]
    car_02.position -= [10, 0, 0]
    scene.get("rx").position = (car_01.position + [0, 0, 0])
    scene.get("rx2").position = (car_02.position + [0, 0, 0])